### Import Library

In [122]:
# 이 중에서 안깔린 라이브러리가 있을 수 있음

import os
from typing import List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt
import pywt

### Load Data

In [123]:
path = "../../../data/"

train = pd.read_csv(path+"train.csv").assign(_type="train")
test = pd.read_csv(path+"test.csv").assign(_type="test")
submission = pd.read_csv(path+"test.csv")
df = pd.concat([train, test], axis=0)

##### 원래 베이스라인 코드 부분

In [124]:
# # HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
# file_names: List[str] = [
#     f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
# ]

# # 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
# file_dict: Dict[str, pd.DataFrame] = {
#     f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
# }

# for _file_name, _df in tqdm(file_dict.items()):
#     # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
#     _rename_rule = {
#         col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
#         for col in _df.columns
#     }
#     _df = _df.rename(_rename_rule, axis=1)
#     df = df.merge(_df, on="ID", how="left")

##### 밑의 한 셀이 바꾼 부분 (이 바꾼 부분을 전제로 뒤의 코드가 실행됨.)

In [125]:
# HOURLY_로 시작하는 모든 csv 파일 불러오기
prefixes = ["HOURLY_NETWORK-DATA_",
            "HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD",
            "HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX",
            "HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE",
            "HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL",
            "HOURLY_MARKET-DATA_OPEN-INTEREST_ALL_EXCHANGE_ALL_SYMBOL",
            "HOURLY_MARKET-DATA_TAKER-BUY-SELL-STATS_ALL_EXCHANGE"]

file_names = [
    f for f in os.listdir(path)
    if any(f.startswith(prefix) for prefix in prefixes) and f.endswith(".csv")
]
display(file_names)
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(path+f) for f in file_names
}


for _file_name, _df in tqdm(file_dict.items()):
    _rename_rule = {
        col: f"{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


['HOURLY_NETWORK-DATA_BLOCK-BYTES.csv',
 'HOURLY_NETWORK-DATA_BLOCK-COUNT.csv',
 'HOURLY_NETWORK-DATA_UTXO-COUNT.csv',
 'HOURLY_NETWORK-DATA_DIFFICULTY.csv',
 'HOURLY_NETWORK-DATA_ADDRESSES-COUNT.csv',
 'HOURLY_NETWORK-DATA_HASHRATE.csv',
 'HOURLY_NETWORK-DATA_TOKENS-TRANSFERRED.csv',
 'HOURLY_MARKET-DATA_OPEN-INTEREST_ALL_EXCHANGE_ALL_SYMBOL.csv',
 'HOURLY_NETWORK-DATA_BLOCKREWARD.csv',
 'HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE.csv',
 'HOURLY_MARKET-DATA_TAKER-BUY-SELL-STATS_ALL_EXCHANGE.csv',
 'HOURLY_NETWORK-DATA_FEES.csv',
 'HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL.csv',
 'HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX.csv',
 'HOURLY_NETWORK-DATA_FEES-TRANSACTION.csv',
 'HOURLY_NETWORK-DATA_SUPPLY.csv',
 'HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD.csv',
 'HOURLY_NETWORK-DATA_BLOCK-INTERVAL.csv',
 'HOURLY_NETWORK-DATA_VELOCITY.csv',
 'HOURLY_NETWORK-DATA_TRANSACTIONS-COUNT.csv']

100%|██████████| 20/20 [00:00<00:00, 27.67it/s]


##### 결측치 (아거도 추가된 부분)

In [126]:
df_new = df.drop(["ID", "target", "_type"], axis = 1)

# 결측치 확인
missing_data = df_new.isnull().sum()

# 결측치 비율 확인
missing_ratio = df_new.isnull().mean()


display(missing_data.sort_values(ascending=False))
display(missing_ratio.sort_values(ascending=False))

volume                         2792
close                          2792
block_bytes                      54
funding_rates                    32
taker_buy_sell_ratio             31
taker_sell_ratio                 31
taker_buy_ratio                  31
taker_sell_volume                31
taker_buy_volume                 31
supply_new                       28
supply_total                     28
transactions_count_mean          28
fees_block_mean_usd              24
fees_transaction_mean            24
fees_transaction_mean_usd        24
tokens_transferred_mean          24
fees_reward_percent              24
block_interval                   24
difficulty                       24
fees_block_mean                  24
coinbase_premium_gap              6
coinbase_premium_index            6
transactions_count_total          4
open_interest                     4
block_count                       4
utxo_count                        1
addresses_count_active            0
velocity_supply_total       

volume                         0.241690
close                          0.241690
block_bytes                    0.004675
funding_rates                  0.002770
taker_buy_sell_ratio           0.002684
taker_sell_ratio               0.002684
taker_buy_ratio                0.002684
taker_sell_volume              0.002684
taker_buy_volume               0.002684
supply_new                     0.002424
supply_total                   0.002424
transactions_count_mean        0.002424
fees_block_mean_usd            0.002078
fees_transaction_mean          0.002078
fees_transaction_mean_usd      0.002078
tokens_transferred_mean        0.002078
fees_reward_percent            0.002078
block_interval                 0.002078
difficulty                     0.002078
fees_block_mean                0.002078
coinbase_premium_gap           0.000519
coinbase_premium_index         0.000519
transactions_count_total       0.000346
open_interest                  0.000346
block_count                    0.000346


In [127]:
# 1. 결측치 처리 - 앞/뒤 보간법 (forward/backward fill)

# df_ffill = df_new.fillna(method='ffill')  # 앞의 값을 사용하여 결측치 채움
# df_bfill = df_new.fillna(method='bfill')  # 뒤의 값을 사용하여 결측치 채움

# 2. 결측치 처리 - 선형 보간법 (linear interpolation)
df_interpolated = df_new.interpolate(method='linear')

# 3. 결측치 처리 - 평균값으로 대체
# df_mean_filled = df_new.fillna(eda_df.mean())

df.update(df_interpolated)

# 결측치 처리 후 확인
print(df_interpolated.isnull().sum())

block_bytes                    0
block_count                    0
utxo_count                     0
difficulty                     0
addresses_count_active         0
addresses_count_sender         0
addresses_count_receiver       0
hashrate                       0
tokens_transferred_total       0
tokens_transferred_mean        0
tokens_transferred_median      0
open_interest                  0
blockreward                    0
blockreward_usd                0
funding_rates                  0
taker_buy_volume               0
taker_sell_volume              0
taker_buy_ratio                0
taker_sell_ratio               0
taker_buy_sell_ratio           0
fees_block_mean                0
fees_block_mean_usd            0
fees_total                     0
fees_total_usd                 0
fees_reward_percent            0
long_liquidations              0
short_liquidations             0
long_liquidations_usd          0
short_liquidations_usd         0
coinbase_premium_gap           0
coinbase_p

### Feature engineering

만약 데이터를 로드할 때, (제공 받은)베이스라인 코드에서와 같이 거래소와 암호화폐 종류를 불러오도록 설정을 했다면

아래 코드는 작동하지 않음

거래소와 암호화폐 종류를 고려하지 않은 코드임을 유의.

(수정을 해야할 것 같음)

In [128]:
# 전체 독립변수에서 내가 사용할 독립변수 고르기


# 전체 feature
features = df.columns
# 내가 쓸 feature
fe = ['ID', 'target', '_type', 'funding_rates', 'open_interest',
       'taker_buy_volume', 'taker_sell_volume', 'taker_buy_ratio',
       'taker_sell_ratio', 'taker_buy_sell_ratio', 'difficulty',
       'transactions_count_total', 'transactions_count_mean', 'block_count',
       'fees_transaction_mean', 'fees_transaction_mean_usd',
       'fees_transaction_median', 'fees_transaction_median_usd',
       'fees_block_mean', 'fees_block_mean_usd', 'fees_total',
       'fees_total_usd', 'fees_reward_percent', 'hashrate', 'utxo_count',
       'tokens_transferred_total', 'tokens_transferred_mean',
       'tokens_transferred_median', 'block_interval', 'velocity_supply_total',
       'supply_total', 'supply_new', 'addresses_count_active',
       'addresses_count_sender', 'addresses_count_receiver',
       'blockreward', 'blockreward_usd', 'buy_sell_volume_ratio']

# new_features에는 내가 사용하지 않을 feature들이 담기고 나중에 drop됨. 때문에, 이름이 좀 잘못된 듯
new_features = [x for x in features if x not in fe]

In [129]:
# eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["taker_buy_volume"] - df["taker_sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["taker_buy_volume"] - df["taker_sell_volume"]),
    buy_sell_volume_ratio=df["taker_buy_volume"] / (df["taker_sell_volume"] + 1),
)

# category, continuous 열을 따로 할당해둠
category_cols = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols = [_ for _ in (df.columns)[3:] if _ not in ["ID", "target", "_type"]] + [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
]

* 🔥 피처엔지니어링 함수 추가

In [131]:
# 날짜 관련 생성
def make_date_features(df: pd.DataFrame, date_column: str) -> Tuple[pd.DataFrame]:
    """
    입력된 데이터프레임의 특정 날짜 열을 기준으로 연도, 월, 주, 요일, 시간을 추출하여 새로운 피처로 추가.
    Args:
        df (pd.DataFrame): 날짜 컬럼을 포함하는 데이터프레임.
        date_column (str): 날짜 정보가 담긴 열 이름.
    Returns:
        pd.DataFrame: 날짜 관련 피처가 추가된 데이터프레임 반환.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    df['year'] = df[date_column].dt.year  # 연도
    df['month'] = df[date_column].dt.month  # 월 
    df['week'] = df[date_column].dt.isocalendar().week  # 주
    df['day_of_week'] = df[date_column].dt.dayofweek  # 요일 
    df['hour'] = df[date_column].dt.hour  # 시간 
    date_columns = [date_column, 'year', 'month', 'week', 'day_of_week', 'hour']
    return df

In [132]:
# 변동성, 차분 피처 생성 함수 
def make_diff_change_feature(
    df: pd.DataFrame, 
    columns_list: List[str]
) -> pd.DataFrame:
    """
    주어진 변수들에 대한 변동성, 차분 피처를 생성하고, 결측값을 처리하는 함수.
    Args:
        df (pd.DataFrame): 데이터를 포함한 데이터프레임.
        columns_list (List[str]): 변동성과 차분 피처를 추가하고자 하는 열 이름 목록.
    Returns:
        pd.DataFrame: 변동성과 차분 피처가 추가된 데이터프레임.
    """  
    new_features = {}  # 새로 추가할 열을 저장할 딕셔너리
    for col in columns_list:
        if col in df.columns:
            pct_change_col = f'{col}_pct_change'
            diff_col = f'{col}_diff'
            new_features[pct_change_col] = df[col].pct_change(fill_method=None)
            new_features[diff_col] = df[col].diff()
        else:
            print(f"Error: Cannot find '{col}' column")
    new_features_df = pd.DataFrame(new_features, index=df.index) # 새 피처를 데이터프레임으로
    new_features_df = new_features_df.ffill().bfill()  # 결측값 처리 (고정: ffill, bfill)
    df = pd.concat([df, new_features_df], axis=1)  # 기존 데이터프레임에 추가
    return df

In [133]:
# 롱/숏 비율
def make_longshort_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str
) -> pd.DataFrame:
    """
    롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 롱/숏 비율이 저장된 새로운 열이 추가된 데이터프레임
    """
    df['long_short_ratio'] = df[long_col] / (df[short_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [134]:
# 청산/거래량 비율
def make_liquidation_to_volume_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str, 
    buy_volume_col: str, 
    sell_volume_col: str
) -> pd.DataFrame:
    """
    청산/거래량 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
        buy_volume_col (str): 매수 거래량이 저장된 열 이름
        sell_volume_col (str): 매도 거래량이 저장된 열 이름
    Returns:
        pd.DataFrame: 청산/거래량 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_to_volume_ratio'] = (
        (df[long_col] + df[short_col]) / 
        (df[buy_volume_col] + df[sell_volume_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    )
    return df

In [135]:
# 청산된 USD 롱/숏 비율
def make_liquidation_usd_ratio_feature(
    df: pd.DataFrame, 
    long_usd_col: str, 
    short_usd_col: str
) -> pd.DataFrame:
    """
    청산된 USD 롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_usd_col (str): 롱(liquidations) USD 데이터가 저장된 열 이름
        short_usd_col (str): 숏(liquidations) USD 데이터가 저장된 열 이름 
    Returns:
        pd.DataFrame: 청산된 USD 롱/숏 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_usd_ratio'] = df[long_usd_col] / (df[short_usd_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [136]:
# 펀딩 비율과 롱/숏 포지션 차이 곱
def make_funding_rate_position_change_feature(
    df: pd.DataFrame, 
    funding_rate_col: str, 
    long_liquidations_col: str, 
    short_liquidations_col: str
) -> pd.DataFrame:
    """
    펀딩 비율과 롱/숏 포지션 차이를 곱하여 포지션 변화를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        funding_rate_col (str): 펀딩 비율 데이터가 저장된 열 이름
        long_liquidations_col (str): 롱 청산 데이터가 저장된 열 이름
        short_liquidations_col (str): 숏 청산 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 펀딩 비율에 따른 포지션 변화를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['funding_rate_position_change'] = df[funding_rate_col] * (
        df[long_liquidations_col] - df[short_liquidations_col]
    )
    return df

In [137]:
# 프리미엄 갭과 프리미엄 인덱스의 차이
def make_premium_diff_feature(
    df: pd.DataFrame, 
    premium_gap_col: str, 
    premium_index_col: str
) -> pd.DataFrame:
    """
    프리미엄 갭과 프리미엄 인덱스의 차이를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        premium_gap_col (str): 프리미엄 갭 데이터가 저장된 열 이름
        premium_index_col (str): 프리미엄 인덱스 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 프리미엄 갭과 프리미엄 인덱스의 차이를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['premium_diff'] = df[premium_gap_col] - df[premium_index_col]
    return df


In [138]:
# 해시레이트와 난이도 간의 비율
def make_hashrate_to_difficulty_feature(
    df: pd.DataFrame, 
    hashrate_col: str, 
    difficulty_col: str
) -> pd.DataFrame:
    """
    해시레이트와 난이도 간의 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        hashrate_col (str): 해시레이트 데이터가 저장된 열 이름
        difficulty_col (str): 난이도 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 해시레이트와 난이도 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['hashrate_to_difficulty'] = df[hashrate_col] / (df[difficulty_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [139]:
# 공급 변화율
def make_supply_change_rate_feature(
    df: pd.DataFrame, 
    new_supply_col: str, 
    total_supply_col: str
) -> pd.DataFrame:
    """
    공급 변화율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        new_supply_col (str): 새로운 공급량 데이터가 저장된 열 이름
        total_supply_col (str): 총 공급량 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 공급 변화율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['supply_change_rate'] = df[new_supply_col] / (df[total_supply_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [140]:
def make_multiple_rolling_features(df, exclude_columns_list):
    """
    여러 윈도우 크기(6, 12, 24, 48)로 이동평균 피처를 생성하는 함수
    
    Parameters:
    - df: 입력 데이터프레임
    - exclude_columns_list: 이동평균을 적용하지 않을 열 이름이 담긴 리스트
    
    Returns:
    - 이동평균 피처가 추가된 데이터프레임
    """
    rolling_features = {}

    # 윈도우 크기별로 변수명 리스트 생성
    rolling_columns_dict = {window: [] for window in [6, 12, 24, 48]}

    for col in df.columns:
        if col not in exclude_columns_list:
            for window in [6, 12, 24, 48]:
                new_col_name = f'{col}_rolling_mean_{window}h'
                rolling_features[new_col_name] = df[col].rolling(window=window).mean()
                # 윈도우 크기에 맞는 리스트에 변수명 저장
                rolling_columns_dict[window].append(new_col_name)

    # 새로운 피처를 한 번에 추가
    rolling_features_df = pd.DataFrame(rolling_features, index=df.index)
    df = pd.concat([df, rolling_features_df], axis=1)
    return df

In [141]:
# 적용 
df = make_date_features(df, 'ID') # 날짜 피처
df = make_diff_change_feature(df, ['open_interest']) # 변동성, 차분 피처 
df = make_longshort_ratio_feature(df, 'long_liquidations', 'short_liquidations') # 롱/숏 비율
df = make_liquidation_to_volume_ratio_feature(df, 'long_liquidations', 'short_liquidations', 'taker_buy_volume', 'taker_sell_volume') # 청산/거래량 비율
df = make_liquidation_usd_ratio_feature(df, 'long_liquidations_usd', 'short_liquidations_usd') # 청산된 USD 롱/숏 비율
df = make_funding_rate_position_change_feature(df, 'funding_rates', 'long_liquidations_usd', 'short_liquidations_usd') # 펀딩 비율과 롱/숏 포지션 차이 곱
df = make_premium_diff_feature(df, 'coinbase_premium_gap', 'coinbase_premium_index') # 프리미엄 갭과 프리미엄 인덱스의 차이
df = make_hashrate_to_difficulty_feature(df, 'hashrate', 'difficulty') # 해시레이트와 난이도 간의 비율
df = make_supply_change_rate_feature(df, 'supply_new', 'supply_total') # 공급 변화율
df = make_multiple_rolling_features(df, ['ID', 'target', '_type'] + ['year', 'month', 'week', 'day_of_week', 'hour'])

rolling_columns = price_columns = [col for col in df.columns if 'rolling' in col]
conti_cols = conti_cols + ['open_interest_pct_change', 'open_interest_diff', 'long_short_ratio', 'liquidation_to_volume_ratio', 'liquidation_usd_ratio', 'funding_rate_position_change', 'premium_diff', 'hashrate_to_difficulty', 'supply_change_rate'] + rolling_columns

In [143]:
# 나머지 위의 피처엔지니어링 함수들을 실행한다.

conti_cols = list(set(conti_cols) - set(rolling_columns))

# 모든 수치형 컬럼에 대한 지수이동평균 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"_moving_avg_7"] = EMA(df, c, 7)

# 모든 수치형 컬럼에 대한 Wavelet transform을 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"WT"] = WT(df, c)

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_4283/4175310149.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_4283/4175310149.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_4283/4175310149.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [144]:
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)
df = df.loc[:, ~df.columns.duplicated()]
df = df.drop(columns=new_features)

train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Training

In [145]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params (베이스라인 코드의 기본 파라미터임)
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/Users/yusolcho/opt/anaconda3/envs/datawrg/lib/python3.8/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4434931506849315, auroc: 0.6441412344399869


In [146]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

/Users/yusolcho/opt/anaconda3/envs/datawrg/lib/python3.8/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


### Inference

In [147]:
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

### Output File Save

In [148]:
submission = submission.assign(target = y_test_pred_class)
submission.to_csv("output.csv", index=False)